# Notebook 8: Unit Tests
## Comprehensive testing of all modules

**Purpose**: Validate all components

**Dependencies**: Notebooks 1-7


## Setup

In [ ]:
!pip install pytest torch numpy pandas -q
import unittest
import pickle
import json
import tempfile
from pathlib import Path
import torch

print("✓ Test framework setup complete")

## Test Configuration

In [ ]:
# Load all modules
modules = {}

try:
    with open('/tmp/utils_module.pkl', 'rb') as f:
        modules['utils'] = pickle.load(f)
except:
    print("⚠️ Utils not found")

try:
    with open('/tmp/job_scraper_module.pkl', 'rb') as f:
        modules['scraper'] = pickle.load(f)
except:
    print("⚠️ Scraper not found")

try:
    with open('/tmp/resume_customizer_module.pkl', 'rb') as f:
        modules['customizer'] = pickle.load(f)
except:
    print("⚠️ Customizer not found")

try:
    with open('/tmp/job_classifier_module.pkl', 'rb') as f:
        modules['classifier'] = pickle.load(f)
except:
    print("⚠️ Classifier not found")

try:
    with open('/tmp/application_agent_module.pkl', 'rb') as f:
        modules['agent'] = pickle.load(f)
except:
    print("⚠️ Agent not found")

print(f"✓ Loaded {len(modules)} modules")

## Test Suites

In [ ]:
class TestUtilities(unittest.TestCase):
    """Test utility functions"""
    
    def setUp(self):
        self.utils = modules.get('utils', {})
    
    def test_utils_loaded(self):
        """Test utilities module loaded"""
        self.assertTrue(len(self.utils) > 0, "Utils module is empty")
    
    def test_text_processing(self):
        """Test text processing functions exist"""
        text_utils = self.utils.get('text_processing', {})
        self.assertIn('clean_text', text_utils)
        self.assertIn('extract_keywords', text_utils)
    
    def test_data_processing(self):
        """Test data processing functions exist"""
        data_utils = self.utils.get('data_processing', {})
        self.assertIn('batch_sequences', data_utils)
        self.assertIn('split_data', data_utils)
    
    def test_file_io(self):
        """Test file I/O functions exist"""
        file_utils = self.utils.get('file_io', {})
        self.assertIn('load_json', file_utils)
        self.assertIn('save_json', file_utils)
    
    def test_similarity_functions(self):
        """Test similarity functions exist"""
        sim_utils = self.utils.get('similarity', {})
        self.assertIn('calculate_similarity', sim_utils)
        self.assertIn('calculate_levenshtein_distance', sim_utils)

print("✓ TestUtilities created")

In [ ]:
class TestJobScraper(unittest.TestCase):
    """Test job scraper functionality"""
    
    def setUp(self):
        scraper_mod = modules.get('scraper', {})
        self.JobScraper = scraper_mod.get('JobScraper')
        self.sample_jobs = scraper_mod.get('sample_jobs', [])
    
    def test_scraper_loaded(self):
        """Test scraper class is loaded"""
        self.assertIsNotNone(self.JobScraper, "JobScraper class not found")
    
    def test_sample_jobs_available(self):
        """Test sample jobs are available"""
        self.assertGreater(len(self.sample_jobs), 0, "No sample jobs found")
    
    def test_job_structure(self):
        """Test job data structure"""
        if self.sample_jobs:
            job = self.sample_jobs[0]
            required_keys = ['title', 'company', 'location', 'skills']
            for key in required_keys:
                self.assertIn(key, job, f"Missing key: {key}")
    
    def test_scraper_instantiation(self):
        """Test scraper can be instantiated"""
        if self.JobScraper:
            scraper = self.JobScraper()
            self.assertIsNotNone(scraper)

print("✓ TestJobScraper created")

In [ ]:
class TestResumeCustomizer(unittest.TestCase):
    """Test resume customizer"""
    
    def setUp(self):
        customizer_mod = modules.get('customizer', {})
        self.ResumeCustomizer = customizer_mod.get('ResumeCustomizer')
        self.sample_resume = customizer_mod.get('sample_resume')
        self.customizer_instance = customizer_mod.get('customizer_instance')
    
    def test_customizer_loaded(self):
        """Test customizer class is loaded"""
        self.assertIsNotNone(self.ResumeCustomizer, "ResumeCustomizer not found")
    
    def test_sample_resume_available(self):
        """Test sample resume is available"""
        self.assertIsNotNone(self.sample_resume, "No sample resume found")
        self.assertGreater(len(self.sample_resume), 0, "Sample resume is empty")
    
    def test_resume_parsing(self):
        """Test resume parsing"""
        if self.customizer_instance:
            parsed = self.customizer_instance.base_resume
            self.assertIsNotNone(parsed)
            self.assertIn('skills', parsed)
            self.assertIn('experience', parsed)
    
    def test_customization_output(self):
        """Test customization produces output"""
        customizer_mod = modules.get('customizer', {})
        customized = customizer_mod.get('customized_sample')
        self.assertIsNotNone(customized)
        self.assertIn('summary', customized)
        self.assertIn('skills', customized)

print("✓ TestResumeCustomizer created")

In [ ]:
class TestJobClassifier(unittest.TestCase):
    """Test job classifier"""
    
    def setUp(self):
        classifier_mod = modules.get('classifier', {})
        self.JobRelevanceClassifier = classifier_mod.get('JobRelevanceClassifier')
        self.results = classifier_mod.get('classification_results')
    
    def test_classifier_loaded(self):
        """Test classifier class is loaded"""
        self.assertIsNotNone(self.JobRelevanceClassifier, "Classifier not found")
    
    def test_classification_results(self):
        """Test classification results exist"""
        self.assertIsNotNone(self.results, "No classification results")
        self.assertIn('statistics', self.results)
        self.assertIn('relevant', self.results)
        self.assertIn('irrelevant', self.results)
    
    def test_classification_stats(self):
        """Test statistics are computed"""
        if self.results:
            stats = self.results['statistics']
            self.assertIn('total_jobs', stats)
            self.assertIn('relevant_count', stats)
            self.assertIn('pass_rate', stats)
            self.assertGreater(stats['total_jobs'], 0)
    
    def test_pass_rate_valid(self):
        """Test pass rate is between 0-1"""
        if self.results:
            pass_rate = self.results['statistics']['pass_rate']
            self.assertGreaterEqual(pass_rate, 0)
            self.assertLessEqual(pass_rate, 1)

print("✓ TestJobClassifier created")

In [ ]:
class TestApplicationAgent(unittest.TestCase):
    """Test application agent"""
    
    def setUp(self):
        agent_mod = modules.get('agent', {})
        self.JobApplicationAgent = agent_mod.get('JobApplicationAgent')
        self.results = agent_mod.get('workflow_results')
    
    def test_agent_loaded(self):
        """Test agent class is loaded"""
        self.assertIsNotNone(self.JobApplicationAgent, "Agent not found")
    
    def test_workflow_results(self):
        """Test workflow produces results"""
        self.assertIsNotNone(self.results, "No workflow results")
        self.assertEqual(self.results['workflow_status'], 'completed')
    
    def test_workflow_summary(self):
        """Test workflow summary is complete"""
        if self.results:
            summary = self.results['summary']
            self.assertIn('jobs_found', summary)
            self.assertIn('jobs_relevant', summary)
            self.assertIn('applications_prepared', summary)
    
    def test_applications_created(self):
        """Test applications are created"""
        if self.results:
            apps = self.results['applications']
            self.assertIsInstance(apps, list)
            if apps:
                app = apps[0]
                self.assertIn('job', app)
                self.assertIn('relevance_score', app)
    
    def test_workflow_log_exists(self):
        """Test workflow logging"""
        if self.results:
            log = self.results['workflow_log']
            self.assertIsInstance(log, list)
            self.assertGreater(len(log), 0)

print("✓ TestApplicationAgent created")

## Run All Tests

In [ ]:
# Create test suite
loader = unittest.TestLoader()
suite = unittest.TestSuite()

suite.addTests(loader.loadTestsFromTestCase(TestUtilities))
suite.addTests(loader.loadTestsFromTestCase(TestJobScraper))
suite.addTests(loader.loadTestsFromTestCase(TestResumeCustomizer))
suite.addTests(loader.loadTestsFromTestCase(TestJobClassifier))
suite.addTests(loader.loadTestsFromTestCase(TestApplicationAgent))

# Run tests
print("\n" + "="*60)
print("RUNNING UNIT TESTS")
print("="*60 + "\n")

runner = unittest.TextTestRunner(verbosity=2)
result = runner.run(suite)

# Print summary
print("\n" + "="*60)
print("TEST SUMMARY")
print("="*60)
print(f"Tests run: {result.testsRun}")
print(f"Successes: {result.testsRun - len(result.failures) - len(result.errors)}")
print(f"Failures: {len(result.failures)}")
print(f"Errors: {len(result.errors)}")

if result.wasSuccessful():
    print("\n✅ ALL TESTS PASSED!")
else:
    print("\n❌ SOME TESTS FAILED")
    for test, traceback in result.failures:
        print(f"\nFailed: {test}")
        print(traceback)

## Test Report

In [ ]:
# Save test report
report = {
    'timestamp': '2025-11-09',
    'total_tests': result.testsRun,
    'passed': result.testsRun - len(result.failures) - len(result.errors),
    'failed': len(result.failures),
    'errors': len(result.errors),
    'success_rate': (result.testsRun - len(result.failures) - len(result.errors)) / result.testsRun if result.testsRun > 0 else 0,
    'test_cases': [
        'TestUtilities (4 tests)',
        'TestJobScraper (4 tests)',
        'TestResumeCustomizer (4 tests)',
        'TestJobClassifier (5 tests)',
        'TestApplicationAgent (5 tests)',
    ],
    'status': 'PASSED' if result.wasSuccessful() else 'FAILED',
}

with open('/tmp/test_report.json', 'w') as f:
    json.dump(report, f, indent=2)

print("\n✓ Test report saved to /tmp/test_report.json")

## Summary

✅ **Notebook 8 Complete**

### Test Coverage:
- Utilities: 4 tests
- Job Scraper: 4 tests
- Resume Customizer: 4 tests
- Job Classifier: 5 tests
- Application Agent: 5 tests

**Total: 22 unit tests**

**Ready for UI (Notebook 9)**